# Key consideration
Avoiding Filter Attributes: In all scripts, we initialize pybgpstream without using any filter attributes. All filtering (e.g., by prefix, ASN, or other criteria) is done within the main loop by checking each record and element.

Processing within Main Loop: By iterating over every element, we ensure that we process all BGP updates and apply our custom logic directly within the loop.

# Counting Announcements and Withdrawals

In [3]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def count_announcements_withdrawals(from_time, until_time, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    announcements = defaultdict(int)
    withdrawals = defaultdict(int)

    for rec in stream.records():
        for elem in rec:
            elem_time = datetime.utcfromtimestamp(elem.time)
            peer_asn = elem.peer_asn
            prefix = elem.fields.get("prefix")

            if prefix is None:
                continue

            if elem.type == 'A':
                announcements[prefix] += 1
            elif elem.type == 'W':
                withdrawals[prefix] += 1

    print("Announcements:")
    for prefix, count in announcements.items():
        print(f"{prefix}: {count}")

    print("\nWithdrawals:")
    for prefix, count in withdrawals.items():
        print(f"{prefix}: {count}")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 00:05:00"
count_announcements_withdrawals(from_time, until_time)

Announcements:
130.137.111.0/24: 94
2a0e:9b43::/48: 817
102.69.52.0/22: 10
156.250.128.0/17: 3
2804:7160::/32: 161
55.58.238.0/24: 93
55.5.236.0/24: 77
67.211.53.0/24: 294
2a12:3fc2:8004::/46: 422
2a0e:b107:1d8::/46: 539
2a06:a005:554::/46: 433
2a10:cc45:625::/48: 18
2a10:cc45:7e0::/48: 14
2a10:cc45:438::/48: 13
2602:feda:bf0::/46: 447
202.45.88.0/24: 128
203.145.74.0/24: 128
203.145.78.0/24: 128
102.214.120.0/22: 1424
84.205.78.0/24: 32
2001:7fb:fe03::/48: 27
2001:7fb:fe01::/48: 22
2001:7fb:fe15::/48: 29
84.205.76.0/24: 37
84.205.67.0/24: 36
64.68.236.0/22: 149
84.205.79.0/24: 26
84.205.65.0/24: 19
58.147.189.0/24: 53
2001:7fb:fe18::/48: 23
2602:ffe4:c17::/48: 53
2604:3b80::/32: 85
168.196.156.0/24: 63
180.149.252.0/24: 28
180.149.253.0/24: 56
182.239.49.0/24: 28
182.239.53.0/24: 56
2606:7540:5::/48: 18
177.70.26.0/24: 27
113.20.140.0/24: 57
113.20.137.0/24: 57
113.20.136.0/24: 57
202.153.26.0/24: 57
202.153.23.0/24: 57
202.153.28.0/24: 57
202.153.17.0/24: 57
202.153.21.0/24: 57
202.1

# Detecting Route Flaps

In [4]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def detect_route_flaps(from_time, until_time, flap_threshold=3, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    prefix_events = defaultdict(list)

    for rec in stream.records():
        for elem in rec:
            elem_time = datetime.utcfromtimestamp(elem.time)
            timestamp = elem.time
            prefix = elem.fields.get("prefix")

            if prefix is None:
                continue

            prefix_events[prefix].append((timestamp, elem.type))

    # Analyze flaps
    for prefix, events in prefix_events.items():
        events.sort()
        flap_count = 0
        last_event_type = None

        for timestamp, event_type in events:
            if last_event_type and last_event_type != event_type:
                flap_count += 1
            last_event_type = event_type

        if flap_count >= flap_threshold:
            print(f"Prefix {prefix} flapped {flap_count} times.")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
detect_route_flaps(from_time, until_time)


Prefix 130.137.111.0/24 flapped 60 times.
Prefix 2a0e:9b43::/48 flapped 1172 times.
Prefix 102.69.52.0/22 flapped 6 times.
Prefix 2804:7160::/32 flapped 154 times.
Prefix 55.58.238.0/24 flapped 175 times.
Prefix 55.5.236.0/24 flapped 177 times.
Prefix 67.211.53.0/24 flapped 522 times.
Prefix 2a12:3fc2:8004::/46 flapped 34 times.
Prefix 2a0e:b107:1d8::/46 flapped 680 times.
Prefix 2a06:a005:554::/46 flapped 28 times.
Prefix 2a10:cc45:625::/48 flapped 11 times.
Prefix 2a10:cc45:7e0::/48 flapped 11 times.
Prefix 2a10:cc45:438::/48 flapped 11 times.
Prefix 2602:feda:bf0::/46 flapped 732 times.
Prefix 2a05:da00::/29 flapped 57 times.
Prefix 130.137.86.0/24 flapped 51 times.
Prefix 202.45.88.0/24 flapped 10 times.
Prefix 203.145.74.0/24 flapped 10 times.
Prefix 203.145.78.0/24 flapped 10 times.
Prefix 2606:2800:e004::/48 flapped 254 times.
Prefix 2a10:cc45:68f::/48 flapped 9 times.
Prefix 2a10:cc45:5e2::/48 flapped 11 times.
Prefix 2a10:cc45:5ca::/48 flapped 11 times.
Prefix 2a02:16c8:6000::

# Monitoring AS Path Changes

In [5]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def monitor_as_path_changes(from_time, until_time, target_prefixes=None, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    prefix_as_paths = defaultdict(set)

    for rec in stream.records():
        for elem in rec:
            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            # If target_prefixes are specified, filter prefixes
            if target_prefixes and prefix not in target_prefixes:
                continue

            as_path_str = elem.fields.get('as-path', "")
            as_path = tuple(as_path_str.split())

            if as_path:
                if as_path not in prefix_as_paths[prefix]:
                    print(f"New AS path for {prefix}: {' '.join(as_path)}")
                    prefix_as_paths[prefix].add(as_path)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 02:00:00"
target_prefixes = ['192.0.2.0/24', '198.51.100.0/24']
monitor_as_path_changes(from_time, until_time, target_prefixes)

# Analyzing Community Attributes

In [ ]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def analyze_communities(from_time, until_time, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    community_counts = defaultdict(int)

    for rec in stream.records():
        for elem in rec:
            communities = elem.fields.get('communities', [])
            for community in communities:
                community_str = f"{community[0]}:{community[1]}"
                community_counts[community_str] += 1

    print("Community Attributes:")
    for community, count in community_counts.items():
        print(f"{community}: {count}")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
analyze_communities(from_time, until_time)

# Detecting BGP Hijacks

In [1]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def detect_bgp_hijacks(from_time, until_time, prefix_origin_mapping, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    hijack_events = []

    for rec in stream.records():
        for elem in rec:
            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            # Check if the prefix is one we're monitoring
            if prefix not in prefix_origin_mapping:
                continue

            as_path_str = elem.fields.get('as-path', "")
            as_path = as_path_str.split()

            if not as_path:
                continue

            origin_asn = as_path[-1]
            expected_asn = prefix_origin_mapping[prefix]

            if origin_asn != expected_asn:
                event = {
                    'timestamp': datetime.utcfromtimestamp(elem.time).isoformat(),
                    'prefix': prefix,
                    'expected_origin': expected_asn,
                    'observed_origin': origin_asn,
                    'as_path': as_path_str,
                    'peer_asn': elem.peer_asn,
                    'collector': rec.collector
                }
                hijack_events.append(event)
                print(f"Possible hijack detected for {prefix}: expected {expected_asn}, observed {origin_asn}")

    # Output the hijack events
    print("\nHijack Events:")
    for event in hijack_events:
        print(event)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
# Define the expected origin ASNs for prefixes
prefix_origin_mapping = {
    '192.0.2.0/24': '64500',
    '198.51.100.0/24': '64501',
    # Add more prefixes and their expected origin ASNs as needed
}
detect_bgp_hijacks(from_time, until_time, prefix_origin_mapping)



Hijack Events:


# Analyzing AS Path Prepending

In [2]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def analyze_as_path_prepending(from_time, until_time, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    prepending_counts = defaultdict(int)

    for rec in stream.records():
        for elem in rec:
            if elem.type != 'A':
                continue

            as_path_str = elem.fields.get('as-path', "")
            as_path = as_path_str.split()

            if not as_path:
                continue

            # Count AS prepending
            last_asn = None
            consecutive_count = 1
            for asn in as_path:
                if asn == last_asn:
                    consecutive_count += 1
                else:
                    if consecutive_count > 1:
                        prepending_counts[last_asn] += consecutive_count - 1
                    consecutive_count = 1
                last_asn = asn

            # Check for prepending at the end of the path
            if consecutive_count > 1 and last_asn:
                prepending_counts[last_asn] += consecutive_count - 1

    # Report top ASes performing prepending
    print("AS Path Prepending Counts:")
    for asn, count in sorted(prepending_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"ASN {asn}: {count} times")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
analyze_as_path_prepending(from_time, until_time)


AS Path Prepending Counts:
ASN 24534: 47624 times
ASN 4837: 22651 times
ASN 131219: 21904 times
ASN 264525: 10745 times
ASN 53115: 9075 times
ASN 8717: 8140 times
ASN 138211: 7876 times
ASN 45582: 7506 times
ASN 8100: 7102 times
ASN 39010: 6346 times
ASN 2046: 5655 times
ASN 61591: 4366 times
ASN 132556: 4364 times
ASN 13904: 4150 times
ASN 138489: 4110 times
ASN 14080: 3929 times
ASN 264911: 3285 times
ASN 263774: 3086 times
ASN 8359: 2907 times
ASN 139351: 2768 times
ASN 8732: 2727 times
ASN 61449: 2468 times
ASN 14203: 2362 times
ASN 20940: 2287 times
ASN 21575: 2172 times
ASN 267484: 2012 times
ASN 18809: 1973 times
ASN 140947: 1755 times
ASN 132649: 1720 times
ASN 63961: 1659 times
ASN 37225: 1620 times
ASN 262336: 1584 times
ASN 17794: 1572 times
ASN 14299: 1552 times
ASN 4637: 1457 times
ASN 17451: 1404 times
ASN 134012: 1323 times
ASN 394990: 1320 times
ASN 7497: 1292 times
ASN 11172: 1215 times
ASN 135330: 1140 times
ASN 136290: 1101 times
ASN 39386: 1077 times
ASN 4761: 1075 

# Monitoring MOAS (Multiple Origin AS) Conflicts

In [3]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def monitor_moas_conflicts(from_time, until_time, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    prefix_origins = defaultdict(set)
    moas_conflicts = defaultdict(set)

    for rec in stream.records():
        for elem in rec:
            if elem.type != 'A':
                continue

            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            as_path_str = elem.fields.get('as-path', "")
            as_path = as_path_str.split()

            if not as_path:
                continue

            origin_asn = as_path[-1]
            prefix_origins[prefix].add(origin_asn)

            if len(prefix_origins[prefix]) > 1:
                moas_conflicts[prefix] = prefix_origins[prefix]

    # Report MOAS conflicts
    print("MOAS Conflicts Detected:")
    for prefix, origins in moas_conflicts.items():
        origins_list = ', '.join(origins)
        print(f"Prefix {prefix} announced by ASNs: {origins_list}")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 02:00:00"
monitor_moas_conflicts(from_time, until_time)


MOAS Conflicts Detected:
Prefix 2001:dc7:fffe::/48 announced by ASNs: 24406, 24151
Prefix 2001:dc7:ffff::/48 announced by ASNs: 24406, 24151
Prefix 2001:dc7::/48 announced by ASNs: 24406, 24151
Prefix 2001:dc7:1000::/48 announced by ASNs: 24406, 24151
Prefix 2001:dc7:3::/48 announced by ASNs: 24409, 24406
Prefix 2001:dc7:fffb::/48 announced by ASNs: 24409, 24406
Prefix 240e:978:4984::/48 announced by ASNs: 146966, 134756
Prefix 240e:978:4980::/48 announced by ASNs: 146966, 134756
Prefix 2407:d340:7129::/48 announced by ASNs: 142031, 61302
Prefix 2602:808:4002::/48 announced by ASNs: 16509, 398960
Prefix 41.86.37.0/24 announced by ASNs: 36958, 12491
Prefix 2605:4400:5::/48 announced by ASNs: 8001, 40715
Prefix 2620:74:27::/48 announced by ASNs: 211369, 396826
Prefix 2001:500:134::/48 announced by ASNs: 211369, 396826
Prefix 2001:500:121::/48 announced by ASNs: 211369, 396826
Prefix 2001:500:125::/48 announced by ASNs: 396826, 397196
Prefix 195.202.91.0/24 announced by ASNs: 9129, {64512

# Detecting Bogon Prefix Announcements

In [ ]:
import pybgpstream
from collections import defaultdict
from datetime import datetime
import ipaddress

# List of bogon prefixes (simplified example)
BOGON_PREFIXES = [
    ipaddress.ip_network('0.0.0.0/8'),
    ipaddress.ip_network('10.0.0.0/8'),
    ipaddress.ip_network('127.0.0.0/8'),
    ipaddress.ip_network('169.254.0.0/16'),
    ipaddress.ip_network('172.16.0.0/12'),
    ipaddress.ip_network('192.168.0.0/16'),
    ipaddress.ip_network('224.0.0.0/4'),
    ipaddress.ip_network('240.0.0.0/4'),
    # Add IPv6 bogon prefixes as needed
]

def is_bogon(prefix):
    try:
        network = ipaddress.ip_network(prefix)
        for bogon in BOGON_PREFIXES:
            if network.subnet_of(bogon):
                return True
    except ValueError:
        pass
    return False

def detect_bogon_announcements(from_time, until_time, collectors=['route-views.sg']):
    """
    Detects announcements of bogon prefixes.

    :param from_time: Start time for data collection.
    :param until_time: End time for data collection.
    :param collectors: List of collectors to use.
    """
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    bogon_announcements = defaultdict(list)

    for rec in stream.records():
        for elem in rec:
            if elem.type != 'A':
                continue

            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            if is_bogon(prefix):
                timestamp = datetime.utcfromtimestamp(elem.time).isoformat()
                bogon_announcements[prefix].append({
                    'timestamp': timestamp,
                    'peer_asn': elem.peer_asn,
                    'as_path': elem.fields.get('as-path', ""),
                    'collector': rec.collector
                })
                print(f"Bogon prefix announced: {prefix} at {timestamp}")

    # Output bogon announcements
    print("\nBogon Announcements:")
    for prefix, events in bogon_announcements.items():
        print(f"\nPrefix: {prefix}")
        for event in events:
            print(event)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
detect_bogon_announcements(from_time, until_time)


# Monitoring Prefix De-aggregation

In [ ]:
import pybgpstream
from collections import defaultdict
from datetime import datetime
import ipaddress

def monitor_prefix_deaggregation(from_time, until_time, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    super_prefixes = defaultdict(set)
    deaggregation_events = []

    for rec in stream.records():
        for elem in rec:
            if elem.type != 'A':
                continue

            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            try:
                network = ipaddress.ip_network(prefix)
            except ValueError:
                continue

            # Check for super-prefixes
            for supernet_prefix in super_prefixes:
                supernet = ipaddress.ip_network(supernet_prefix)
                if network != supernet and network.subnet_of(supernet):
                    event = {
                        'timestamp': datetime.utcfromtimestamp(elem.time).isoformat(),
                        'super_prefix': supernet_prefix,
                        'more_specific_prefix': prefix,
                        'collector': rec.collector
                    }
                    deaggregation_events.append(event)
                    print(f"De-aggregation detected: {prefix} is a subnet of {supernet_prefix}")
                    break

            # Add the prefix to super_prefixes
            super_prefixes[str(network)] = network

    # Output de-aggregation events
    print("\nDe-aggregation Events:")
    for event in deaggregation_events:
        print(event)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 02:00:00"
monitor_prefix_deaggregation(from_time, until_time)

# Analyzing MED and Local Preference Attributes

In [ ]:
import pybgpstream
from collections import defaultdict
from datetime import datetime
import statistics

def analyze_med_local_pref(from_time, until_time, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    med_values = []
    local_pref_values = []

    for rec in stream.records():
        for elem in rec:
            if elem.type != 'A':
                continue

            update = elem.fields
            med = update.get('med')
            if med is not None:
                try:
                    med_values.append(int(med))
                except ValueError:
                    pass  # Ignore invalid MED values

            local_pref = update.get('local-pref')
            if local_pref is not None:
                try:
                    local_pref_values.append(int(local_pref))
                except ValueError:
                    pass  # Ignore invalid Local Preference values

    # Compute statistics
    if med_values:
        med_avg = statistics.mean(med_values)
        med_min = min(med_values)
        med_max = max(med_values)
        med_stddev = statistics.stdev(med_values) if len(med_values) > 1 else 0
        print("MED Statistics:")
        print(f"Average: {med_avg}, Min: {med_min}, Max: {med_max}, StdDev: {med_stddev}")
    else:
        print("No MED values found.")

    if local_pref_values:
        lp_avg = statistics.mean(local_pref_values)
        lp_min = min(local_pref_values)
        lp_max = max(local_pref_values)
        lp_stddev = statistics.stdev(local_pref_values) if len(local_pref_values) > 1 else 0
        print("\nLocal Preference Statistics:")
        print(f"Average: {lp_avg}, Min: {lp_min}, Max: {lp_max}, StdDev: {lp_stddev}")
    else:
        print("No Local Preference values found.")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
analyze_med_local_pref(from_time, until_time)


# Analyzing BGP Session Resets

In [4]:
import pybgpstream
from collections import defaultdict
from datetime import datetime, timedelta

def analyze_bgp_session_resets(from_time, until_time, reset_threshold=5, collectors=['route-views.sg']):
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    withdrawal_events = defaultdict(list)
    session_resets = []

    for rec in stream.records():
        for elem in rec:
            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            peer_asn = elem.peer_asn
            timestamp = datetime.utcfromtimestamp(elem.time)

            if elem.type == 'W':
                withdrawal_events[peer_asn].append((timestamp, prefix))

    # Analyze for session resets
    for peer, events in withdrawal_events.items():
        events.sort()
        # Group withdrawals that occur within a short time window (e.g., 1 minute)
        window_start = None
        window_events = []

        for event_time, prefix in events:
            if window_start is None:
                window_start = event_time
                window_events.append((event_time, prefix))
            elif (event_time - window_start) <= timedelta(minutes=1):
                window_events.append((event_time, prefix))
            else:
                if len(window_events) >= reset_threshold:
                    session_resets.append({
                        'peer_asn': peer,
                        'start_time': window_start.isoformat(),
                        'end_time': event_time.isoformat(),
                        'withdrawal_count': len(window_events)
                    })
                    print(f"BGP session reset suspected for peer {peer} between {window_start} and {event_time}")

                # Reset window
                window_start = event_time
                window_events = [(event_time, prefix)]

    # Output session reset events
    print("\nBGP Session Reset Events:")
    for reset in session_resets:
        print(reset)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 03:00:00"
analyze_bgp_session_resets(from_time, until_time)

BGP session reset suspected for peer 24115 between 2023-01-01 00:00:00 and 2023-01-01 00:01:03
BGP session reset suspected for peer 24115 between 2023-01-01 00:01:03 and 2023-01-01 00:02:04
BGP session reset suspected for peer 24115 between 2023-01-01 00:02:04 and 2023-01-01 00:03:06
BGP session reset suspected for peer 24115 between 2023-01-01 00:03:06 and 2023-01-01 00:04:08
BGP session reset suspected for peer 24115 between 2023-01-01 00:04:08 and 2023-01-01 00:05:10
BGP session reset suspected for peer 24115 between 2023-01-01 00:05:10 and 2023-01-01 00:06:11
BGP session reset suspected for peer 24115 between 2023-01-01 00:06:11 and 2023-01-01 00:07:13
BGP session reset suspected for peer 24115 between 2023-01-01 00:07:13 and 2023-01-01 00:08:23
BGP session reset suspected for peer 24115 between 2023-01-01 00:08:23 and 2023-01-01 00:09:28
BGP session reset suspected for peer 24115 between 2023-01-01 00:09:28 and 2023-01-01 00:10:29
BGP session reset suspected for peer 24115 between

# Analyzing Convergence Times

In [ ]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def analyze_convergence_times(from_time, until_time, target_prefixes=None, collectors=['route-views.sg']):
    """
    Analyzes BGP convergence times for prefix updates.

    :param from_time: Start time for data collection.
    :param until_time: End time for data collection.
    :param target_prefixes: List of prefixes to analyze; if None, analyze all prefixes.
    :param collectors: List of collectors to use.
    """
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    prefix_update_times = defaultdict(list)

    for rec in stream.records():
        for elem in rec:
            prefix = elem.fields.get("prefix")
            if prefix is None:
                continue

            if target_prefixes and prefix not in target_prefixes:
                continue

            timestamp = elem.time
            prefix_update_times[prefix].append(timestamp)

    # Calculate convergence times
    for prefix, times in prefix_update_times.items():
        first_time = min(times)
        last_time = max(times)
        convergence_time = last_time - first_time
        print(f"Prefix {prefix} convergence time: {convergence_time} seconds")

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 01:00:00"
target_prefixes = ['192.0.2.0/24']
analyze_convergence_times(from_time, until_time, target_prefixes)


# Detecting BGP Route Leaks

In [5]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

# Simplified relationships for illustration purposes
AS_RELATIONSHIPS = {
    '64500': {'providers': ['64501'], 'customers': ['64502']},
    '64501': {'providers': [], 'customers': ['64500']},
    '64502': {'providers': ['64500'], 'customers': []},
    # Add more AS relationships as needed
}

def is_route_leak(as_path):
    """
    Detects if an AS path indicates a route leak based on simplified relationships.

    :param as_path: List of ASNs in the AS path.
    :return: True if a route leak is detected, False otherwise.
    """
    for i in range(len(as_path) - 1):
        asn_current = as_path[i]
        asn_next = as_path[i + 1]
        relationships = AS_RELATIONSHIPS.get(asn_current, {})
        if asn_next in relationships.get('providers', []):
            # Route leak detected: customer announcing to provider
            return True
    return False

def detect_route_leaks(from_time, until_time, collectors=['route-views.sg']):
    """
    Detects potential BGP route leaks.

    :param from_time: Start time for data collection.
    :param until_time: End time for data collection.
    :param collectors: List of collectors to use.
    """
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    route_leak_events = []

    for rec in stream.records():
        for elem in rec:
            if elem.type != 'A':
                continue

            as_path_str = elem.fields.get('as-path', "")
            as_path = as_path_str.split()

            if not as_path or len(as_path) < 2:
                continue

            if is_route_leak(as_path):
                event = {
                    'timestamp': datetime.utcfromtimestamp(elem.time).isoformat(),
                    'as_path': as_path_str,
                    'collector': rec.collector
                }
                route_leak_events.append(event)
                print(f"Potential route leak detected: AS path {as_path_str}")

    # Output route leak events
    print("\nRoute Leak Events:")
    for event in route_leak_events:
        print(event)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 03:00:00"
detect_route_leaks(from_time, until_time)


Route Leak Events:


# Monitoring Updates for Specific ASNs

In [ ]:
import pybgpstream
from collections import defaultdict
from datetime import datetime

def monitor_updates_for_asns(from_time, until_time, target_asns, collectors=['route-views.sg']):
    target_asns = set(str(asn) for asn in target_asns)
    stream = pybgpstream.BGPStream(
        from_time=from_time,
        until_time=until_time,
        record_type="updates",
        collectors=collectors
    )

    updates_involving_asns = []

    for rec in stream.records():
        for elem in rec:
            as_path_str = elem.fields.get('as-path', "")
            as_path = as_path_str.split()

            if not as_path:
                continue

            if target_asns.intersection(as_path):
                update = {
                    'timestamp': datetime.utcfromtimestamp(elem.time).isoformat(),
                    'as_path': as_path_str,
                    'prefix': elem.fields.get("prefix"),
                    'type': elem.type,
                    'collector': rec.collector
                }
                updates_involving_asns.append(update)
                print(f"Update involving target ASN(s): {update}")

    # Output collected updates
    print("\nUpdates Involving Target ASNs:")
    for update in updates_involving_asns:
        print(update)

# Usage
from_time = "2023-01-01 00:00:00"
until_time = "2023-01-01 02:00:00"
target_asns = ['64500', '64501']
monitor_updates_for_asns(from_time, until_time, target_asns)
